# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Modelos-jerárquicos:-Introducción" data-toc-modified-id="Modelos-jerárquicos:-Introducción-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modelos jerárquicos: Introducción</a></div><div class="lev2 toc-item"><a href="#¿Qué-es-un-modelo-jerárquico?" data-toc-modified-id="¿Qué-es-un-modelo-jerárquico?-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>¿Qué es un modelo jerárquico?</a></div><div class="lev2 toc-item"><a href="#Modelo-de-datos,-modelo-de-proceso-y-parámetros-de-modelo" data-toc-modified-id="Modelo-de-datos,-modelo-de-proceso-y-parámetros-de-modelo-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Modelo de datos, modelo de proceso y parámetros de modelo</a></div><div class="lev2 toc-item"><a href="#Simulaciones" data-toc-modified-id="Simulaciones-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Simulaciones</a></div><div class="lev1 toc-item"><a href="#Ejemplo-simple" data-toc-modified-id="Ejemplo-simple-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ejemplo simple</a></div><div class="lev2 toc-item"><a href="#Ejemplo-con-covariables" data-toc-modified-id="Ejemplo-con-covariables-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Ejemplo con covariables</a></div>

data model(s), process model(s), and parameter models

In [ ]:
import pystan
import numpy as np
import geopandas as geopd
import random
import pandas as pd
import pystan
import math
from scipy.stats import bernoulli, uniform, binom
import matplotlib
import matplotlib.pyplot as plt
import functools
from IPython.display import Latex

%matplotlib inline

# Modelos jerárquicos: Introducción
(Ref: Applied Hierarchical Modeling in Ecology Sec. 10.3)

## ¿Qué es un modelo jerárquico?

## Modelo de datos, modelo de proceso y parámetros de modelo

## Simulaciones 

# Ejemplo simple

Simulación con 100 sitios de muestreo de presencia, para alguna especie hipotética, con dos visitas por sitio (i.e. 200 observaciones). La probabilidad de presencia de la especie es 0.8 y su probabilidad de detección es 0.5. 

In [ ]:
random.seed(24)

In [ ]:
M = 100
J = 2

psi = 0.8 # probabilidad de presencia
p = 0.5 # probabilidad de detección

In [ ]:
stats = bernoulli.stats(psi)
z = bernoulli.rvs(psi, size=M)
stats = {'$\psi$': psi, '$\mu$': psi, '$\sigma$': round(math.sqrt(psi*(1-psi)), 2)}


$$z \sim Bernoulli(\psi)$$

{{pd.DataFrame(data=stats, index=['stats'])}}

{{pd.DataFrame(z, columns=["z"]).describe()}}

In [ ]:
y = []
for z_i in z:
    y.append(bernoulli.rvs(p * z_i, size=J))

y = np.array(y)

{{"Sitios ocupados: {}".format(z.sum())}}
{{"Detecciones: {}".format(y.max(axis = 1).sum())}}

$$y \mid z \sim Bernoulli(z*p)$$
$$ z = 0 \implies y = 0$$
$$ z = 1 \implies y \sim Bernoulli(p) $$
{{pd.DataFrame(z, columns=["z"]).describe()}}

In [ ]:
occ_code = """
    data {
        int<lower=1> M; // M >= 1 numero de sitios
        int<lower=0, upper=1> z[M]; // estado de ocupacion z[m] en {0, 1}
        int<lower=1> J;
        int<lower=0, upper=1> y[M, J]; // observaciones y[m]
    }
    
    parameters {
        real<lower=0, upper=1> psi;  // probabilidad de presencia
        real<lower=0, upper=1> p; // chance de detectar
    }
    

    model {
        psi ~ uniform(0,1); // prior
        p ~ uniform(0, 1);  // prior
        z ~ bernoulli(psi);
        
        for (i in 1:M)
            y[i] ~ bernoulli(p * z[i]);
    }
"""

In [ ]:
sm_simple = pystan.StanModel(model_code=occ_code)

In [ ]:
survey_dat = {'M': M,
              'J': J,
              'y': y,
              'z': z}

fit_simple = sm_simple.sampling(data=survey_dat, iter=5000, chains=3)

In [ ]:
fit_simple

In [ ]:
fit_simple.plot()

In [ ]:
## Ejemplo 2 con convariables
J = 3
sim_desc = Latex(r"""
    \begin{eqnarray*}
        vegHt & \sim & uniform(-1, 1) \\
        wind & \sim & uniform(-1, 1) \\
        \psi & \sim & logistic(\beta_0 + \beta_1 vegHt) \\
        p & \sim & logistic(\alpha_0 + \alpha_1 wind) \\
        \\
        z & \sim & Bernoulli(\psi \mid vegHt) \\
        y & \sim & Bernoulli(p \mid z, wind)
    \end{eqnarray*}""")

## Ejemplo con covariables
M = {{M}}, J = {{J}} 

{{sim_desc}}

In [ ]:
def  mi_logistic(x):
    return 1 / (1 + math.exp(-x))

vegHT = uniform.rvs(-1, 2, size=M)
vegHT = np.array(vegHT)
vegHT.sort()

{{pd.DataFrame(vegHT).describe()}}

In [ ]:
beta = [0, 3]

In [ ]:
psi = map(mi_logistic, beta[0] + (beta[1] * vegHT))
psi = np.array(psi)
plt.plot(vegHT, psi)

In [ ]:
from collections import Counter

z = bernoulli.rvs(psi, size=M)

In [ ]:
wind = [uniform.rvs(-1, 2, size=J) for _ in xrange(M)]

alfa = [-2, -3]
p = np.array([map(mi_logistic, alfa[0] + alfa[1] * w_row) for w_row in wind])
theta = [x[0] * x[1] for x in zip(z, p)]
y = [bernoulli.rvs(theta_row, size=J) for theta_row in theta]
y = np.array(y)

In [ ]:
y.max(axis=1).sum()

In [ ]:
model_code = """
   data {
        int<lower=1> M; // M >= 1 numero de sitios
        int<lower=0, upper=1> z[M]; // estado de ocupacion z[m] en {0, 1}
        int<lower=1> J;
        int<lower=0> y[M, J]; // observaciones y[m]
        real vegHt[M];
        real wind[M, J];
    }
    
    parameters {
        real beta[2];  // probabilidad de presencia
        real alfa[2]; // chance de detectar
    }
    
    model {
        real psi;
        real p;
        
        for (i in 1:M) {
            psi =  beta[1] + beta[2] * vegHt[i];
            z[i] ~ bernoulli_logit(psi);
        }

        for (i in 1:M)
            for (j in 1:J) {
                p = z[i] * inv_logit(alfa[1] + alfa[2] * wind[i,j]);
                y[i,j] ~ bernoulli(p);
            }
    }

"""

In [ ]:
sm_complex = pystan.StanModel(model_code=model_code)

In [ ]:
survey_dat = {'M': M,
              'J': J,
              'y': y,
              'z': z,
              'vegHt': vegHT,
              'wind': wind}

In [ ]:
fit_complex = sm_complex.sampling(data=survey_dat, iter=5000, chains=4)

In [ ]:
fit_complex

In [ ]:
fit_simple_2 = sm_simple.sampling(data=survey_dat, iter=5000, chains=4)

In [ ]:
fit_simple_2

In [ ]:
fit_complex.plot()

In [ ]:
fit_simple_2.plot()

In [ ]:
fit_simple.plot()

In [ ]:
grid_32km = geopd.read_file("../SPECIES/repos/snib-middleware/geofiles/niche/grid_32km.json")

In [ ]:
grid_32km.describe()